In [2]:
from dotenv import load_dotenv
from typing import Dict, Any, List
import yfinance as yf
load_dotenv() 

True

ReAct Agent implementation with Supervisor implementation

In [3]:
def get_stock_data(ticker: str) -> Dict[str, Any]:
    """Fetches stock data for a
      given stock ticker using yfinance.
    
    Args:
        ticker (str): The stock ticker
      symbol (e.g., "AAPL", "MSFT").
    
    Returns:
        Dict[str, Any]: A dictionary
      containing selected stock information,
      or an error message if the ticker is invalid.
    """
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        if not info:
            return {"error": f"Could not retrieve data for ticker: {ticker}. It might be invalid."}

        # Extract relevant information
        data = {
            "ticker": ticker,
            "shortName": info.get("shortName", "N/A"),
            "currentPrice": info.get("currentPrice", "N/A"),
            "previousClose": info.get("previousClose", "N/A"),
            "open": info.get("open", "N/A"),
            "dayHigh": info.get("dayHigh", "N/A"),
            "dayLow": info.get("dayLow", "N/A"),
            "volume": info.get("volume", "N/A"),
            "marketCap": info.get("marketCap", "N/A"),
            "currency": info.get("currency", "N/A"),
            "exchange": info.get("exchange", "N/A"),
            "sector": info.get("sector", "N/A"),
            "industry": info.get("industry", "N/A"),
            "longBusinessSummary": info.get("longBusinessSummary", "N/A")
        }
        return data
    except Exception as e:
        return {"error": f"An unexpected error occurred while fetching data for {ticker}: {e}"}


def get_financial_data(ticker: str) -> str:
    """Fetches stock data for a
      given stock ticker using yfinance.
    
    Args:
        ticker (str): The stock ticker
      symbol (e.g., "AAPL", "MSFT").
    
    Returns:
        str: A String of data containing income_statement, 
        cash_flow, balance_sheet, financials, historical_month_data
      or an error message if the ticker is invalid.
    """
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        if not info:
            return {"error": f"Could not retrieve data for ticker: {ticker}. It might be invalid."}

        # Extract relevant information
        data = f"""
            'income_statement': 
            {stock.income_stmt}

            'cash_flow': 
            {stock.cash_flow}

            'balance_sheet': 
            {stock.balance_sheet}

            'financials': 
            {stock.financials}

            'historical_month_data':
            {stock.history(period='1mo')}
        """
        return data
    except Exception as e:
        return {"error": f"An unexpected error occurred while fetching data for {ticker}: {e}"}



def get_news_articles(query: str) -> List[Dict]:
    """Fetch news articles based on query or ticker symbol.
    
    Args:
        query: Search keywords or phrases (str)
            
    Returns:
        List of news articles with title, description, url, etc.
    """
    try:
        from newsapi import NewsApiClient
        import os
        
        newsapi = NewsApiClient(api_key=os.getenv("NEWS_API_KEY"))
        response = newsapi.get_everything(
            q=query,
            language="en",
            sort_by="publishedAt",
            page_size=10
        )
        return response.get("articles", [])
    except Exception as e:
        return {"error": f"Failed to fetch news: {str(e)}"}

First agent is the News agent that fetches the latest news data from the News API. 
The News API is called with the tool get_news_articles.

In [5]:
from langgraph.prebuilt import create_react_agent

news_agent = create_react_agent(
    model="openai:gpt-4.1",
    tools=[get_news_articles],
    prompt=(
        "You are a news agent.\n\n"
        "INSTRUCTIONS:\n"
        "- Assist ONLY with research-related tasks, DO NOT do any math\n"
        "- After you're done with your tasks, respond to the supervisor directly\n"
        "- Respond ONLY with the results of your work, do NOT include ANY other text."
    ),
    name="news_agent",
)

Utility Functions for tracing message passing within the agents

In [6]:
from langchain_core.messages import convert_to_messages


def pretty_print_message(message, indent=False):
    pretty_message = message.pretty_repr(html=True)
    if not indent:
        print(pretty_message)
        return

    indented = "\n".join("\t" + c for c in pretty_message.split("\n"))
    print(indented)


def pretty_print_messages(update, last_message=False):
    is_subgraph = False
    if isinstance(update, tuple):
        ns, update = update
        # skip parent graph updates in the printouts
        if len(ns) == 0:
            return

        graph_id = ns[-1].split(":")[0]
        print(f"Update from subgraph {graph_id}:")
        print("\n")
        is_subgraph = True

    for node_name, node_update in update.items():
        update_label = f"Update from node {node_name}:"
        if is_subgraph:
            update_label = "\t" + update_label

        print(update_label)
        print("\n")

        messages = convert_to_messages(node_update["messages"])
        if last_message:
            messages = messages[-1:]

        for m in messages:
            pretty_print_message(m, indent=is_subgraph)
        print("\n")

In [7]:
for chunk in news_agent.stream(
    {"messages": [{"role": "user", "content": "Get the news about AAPL stock?"}]}
):
    pretty_print_messages(chunk)

Update from node agent:


================================== Ai Message ==================================
Name: news_agent
Tool Calls:
  get_news_articles (call_fC5gMduOveJT9JlN5TDjL0J2)
 Call ID: call_fC5gMduOveJT9JlN5TDjL0J2
  Args:
    query: AAPL stock


Update from node tools:


================================= Tool Message =================================
Name: get_news_articles

[{"source": {"id": null, "name": "Yahoo Entertainment"}, "author": "Chris Kirkham", "title": "Analysis-Musk's record Tesla package will pay him tens of billions even if he misses most goals", "description": "LOS ANGELES (Reuters) -When Tesla directors offered Elon Musk the biggest executive pay package in corporate history in September, it reassured investors...", "url": "https://finance.yahoo.com/news/analysis-musks-record-tesla-package-100309588.html", "urlToImage": "https://media.zenfs.com/en/reuters-finance.com/2c38373972258c1c0c93a0339f5593d1", "publishedAt": "2025-10-09T10:03:09Z", "content": "B

The secont agent is the Stock Agent that fetches the stock data from the Stock API.
The Stock API is called with the tool get_stock_data 

In [8]:
stock_agent = create_react_agent(
    model="openai:gpt-4.1",
    tools=[get_stock_data, get_financial_data],
    prompt=(
        "You are a stock agent.\n\n"
        "INSTRUCTIONS:\n"
        "- Assist ONLY with stock-related tasks\n"
        "- After you're done with your tasks, respond to the supervisor directly\n"
        "- Respond ONLY with the results of your work, do NOT include ANY other text."
    ),
    name="stock_agent",
)

In [10]:
for chunk in stock_agent.stream(
    {"messages": [{"role": "user", "content": "Get the financial data related to AAPL stock"}]}
):
    pretty_print_messages(chunk)

Update from node agent:


================================== Ai Message ==================================
Name: stock_agent
Tool Calls:
  get_financial_data (call_0Tk9muS02apLyIFihAQ8z5kv)
 Call ID: call_0Tk9muS02apLyIFihAQ8z5kv
  Args:
    ticker: AAPL


Update from node tools:


================================= Tool Message =================================
Name: get_financial_data


            'income_statement': 
                                                                  2024-09-30  \
Tax Effect Of Unusual Items                         0.000000e+00   
Tax Rate For Calcs                                  2.410000e-01   
Normalized EBITDA                                   1.346610e+11   
Net Income From Continuing Operation Net Minori...  9.373600e+10   
Reconciled Depreciation                             1.144500e+10   
Reconciled Cost Of Revenue                          2.103520e+11   
EBITDA                                              1.346610e+11   
EBIT                

In [11]:
for chunk in stock_agent.stream(
    {"messages": [{"role": "user", "content": "Get the stock information for AAPL"}]}
):
    pretty_print_messages(chunk)

Update from node agent:


================================== Ai Message ==================================
Name: stock_agent
Tool Calls:
  get_stock_data (call_sr8T6DzBRgGZwcpmSYMbxMbo)
 Call ID: call_sr8T6DzBRgGZwcpmSYMbxMbo
  Args:
    ticker: AAPL


Update from node tools:


================================= Tool Message =================================
Name: get_stock_data

{"ticker": "AAPL", "shortName": "Apple Inc.", "currentPrice": 254.04, "previousClose": 258.06, "open": 257.9, "dayHigh": 258.0, "dayLow": 253.1402, "volume": 38191297, "marketCap": 3770052509696, "currency": "USD", "exchange": "NMS", "sector": "Technology", "industry": "Consumer Electronics", "longBusinessSummary": "Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising Ai

The json parser Agent

In [31]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class NewsSentiment(BaseModel):
    news_headline: str = Field(..., description="News Headline")
    time: str = Field(..., description="Time of the news. Friendly format like 1 week ago, 1 month ago, 1year ago")
    sentiment: float = Field(..., description="News rating between -1 and +1")

class SentimentAnalysis(BaseModel):
    key_words: List[str] = Field(..., description="Key words related to finance from recent news")
    news_sentiment: List[NewsSentiment] = Field(..., description="A list of news headlines and its sentiment")
    overall_sentiment_rating: float = Field(..., description="Overall sentiment of the company in the news, rated between -10 and +10")
    reasoning: str = Field(..., description="A one line reason for assigning the overall sentiment rating")

class StockRecommendation(BaseModel):
    recommendation: str = Field(..., description="Recommendation for the stock. Strong Buy, Buy, Hold, Sell, Strong Sell")
    reasoning: List[str] = Field(..., description="3 reasons for assigning the recommendation")
    price_prediction: float = Field(..., description="Predicted price of the stock")
    price_prediction_percentage: float = Field(..., description="Predicted price percentage change of the stock")

class RiskAssessment(BaseModel):
    market_risk: str = Field(..., description="Risk level of the stock. Low, Medium, High")
    volatility: str = Field(..., description="Volatility of the stock. Low, Medium, High")
    growth_potential: str = Field(..., description="Growth potential of the stock. Low, Medium, High")

class StockOverview(BaseModel):
    company_overview: str = Field(..., description="Financial summary of the company in a short paragraph")
    stock_recommendation: StockRecommendation = Field(..., description="Stock recommendation")
    risk_assessment: RiskAssessment = Field(..., description="Risk assessment")
    sentiment_analysis: SentimentAnalysis = Field(..., description="Sentiment analysis")

parser = JsonOutputParser(pydantic_object=StockOverview)

# System message
sys_msg = f"""
You are an expert financial analyst. Analyze the stock data and news, 
then respond with a comprehensive analysis in JSON format.

Response must be a valid JSON object matching this schema:
{parser.get_format_instructions()}

Guidelines:
- Be objective and data-driven
- Include both technical and fundamental analysis
- Consider recent news impact
- Provide clear, actionable recommendations
- overall sentiment rating score should reflect certainty (-10 to +10)
"""


In [32]:
json_parser_agent = create_react_agent(
    model="openai:gpt-4.1",
    tools=[],
    prompt=(sys_msg),
    name="json_parser_agent",
)

The 2 agents should be orchastrated by a parent agent.

In [43]:
from langgraph_supervisor import create_supervisor
from langchain.chat_models import init_chat_model

supervisor = create_supervisor(
    model=init_chat_model("openai:gpt-4.1"),
    agents=[stock_agent, news_agent, json_parser_agent],
    prompt=(
        "You are a supervisor managing three agents:\n"
        "- a stock agent. Assign stock-related tasks to this agent\n"
        "- a news agent. Assign news-related tasks to this agent\n"
        "- a json parser agent. Assign json parsing tasks to this agent\n"
        "Assign work to one agent at a time, do not call agents in parallel.\n"
        "Do not do any work yourself."
    ),
    add_handoff_back_messages=True,
    output_mode="full_history",
).compile()

In [39]:
from IPython.display import display, Image

mermaid_code = supervisor.get_graph().draw_mermaid()
print(mermaid_code)

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	supervisor(supervisor)
	stock_agent(stock_agent)
	news_agent(news_agent)
	json_parser_agent(json_parser_agent)
	__end__([<p>__end__</p>]):::last
	__start__ --> supervisor;
	json_parser_agent --> supervisor;
	news_agent --> supervisor;
	stock_agent --> supervisor;
	supervisor -.-> __end__;
	supervisor -.-> json_parser_agent;
	supervisor -.-> news_agent;
	supervisor -.-> stock_agent;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [45]:
from langchain_core.messages import HumanMessage, SystemMessage

def prepare_human_message(ticker: str) -> str:
    return f"""
Analyze this stock data, financial information, recent news and provide your analysis in the requested JSON format.

STOCK DATA for {ticker.upper()}:

Provide your analysis in the exact JSON format specified, with no additional text.
"""


for chunk in supervisor.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": prepare_human_message("AAPL"),
            }
        ]
    },
):
    pretty_print_messages(chunk, last_message=True)

Update from node supervisor:


================================= Tool Message =================================
Name: transfer_to_stock_agent

Successfully transferred to stock_agent


Update from node stock_agent:


================================= Tool Message =================================
Name: transfer_back_to_supervisor

Successfully transferred back to supervisor


Update from node supervisor:


================================= Tool Message =================================
Name: transfer_to_news_agent

Successfully transferred to news_agent


Update from node news_agent:


================================= Tool Message =================================
Name: transfer_back_to_supervisor

Successfully transferred back to supervisor


Update from node supervisor:


================================= Tool Message =================================
Name: transfer_to_json_parser_agent

Successfully transferred to json_parser_agent


Update from node json_parser_agent:


==========